In [1]:
#Import the necessary libraries for navigating the US Census API
import requests
import pandas as pd
import ast
import re
import numpy as np

In [2]:
#Creates a dictionary of codes and corresponding titles for the 2015 American Community Survey
census_codes_2015 = {
    'Total_Pop': 'B01003_001E',
    'Total_Males': 'B01001_002E',
    'Total_Females': 'B01001_026E',
    'Median_Age': 'B01002_001E',
    'Total_Veterans': 'B21001_002E',
    'Total_White_People': 'B02001_002E',
    'Total_Afr_Am_People': 'B02001_003E',
    'Total_Asian': 'B02001_005E',
    'Total_American_Ind_Alk_Ntv': 'B02001_004E',
    'Total_Cuban_Origin': 'C03001_006E',
    'Total_Puerto_Rican_Origin':'C03001_005E',
    'Total_Dominican_Origin':'C03001_007E',
    'Total_Mexican_Origin': 'C03001_004E',
    'Total_Native_US':'B05001_002E',
    'Total_Non_Citizens': 'B05001_006E',
    'Total_Foreign_Born': 'B05012_003E',
    'Total_Children': 'B09001_001E',
    'Total_Married_Households': 'B11001_003E',
    'Total_Less_Highschool': 'B06009_002E',
    'Total_Bachelor_Holders': 'B06009_005E',
    'Total_STEM': 'B15012_009E',
    'Total_Business_Degree': 'B15012_010E',
    'Total_Labor_Force_Eligible': 'B28007_002E', #code changed in 2015
    'Total_Employed': 'B28007_003E',
    'Total_Unemployed': 'B28007_009E',
    'Median_Household_Income': 'B19013_001E',
    'Median_Income': 'B06011_001E',
    'Below_Poverty_Level_LTM': 'B17001_002E',
    'Total_Wealthy_Households': 'B19001_017E', #Earns $200k or more
    'Gini_Index': 'B19083_001E',
    'Median_Age_Of_Worker': 'B23013_001E',
    'Total_Workers': 'B24050_001E',
    'Total_Working_Agricult': 'B24050_002E',
    'Total_Working_Construction': 'B24050_005E',
    'Total_Working_Manufacturing': 'B24050_006E',
    'Total_Working_Retail': 'B24050_008E',
    'Total_Working_Transportation': 'B24050_009E',
    'Total_Working_Finance': 'B24050_013E',
    'Total_Working_Education': 'B24050_021E',
    'Total_Working_Health': 'B24050_022E',
    'Total_Working_Food': 'B24050_025E',
    'Total_Working_Public_Admin': 'B24050_027E',
    'Total_Working_Information': 'B24050_012E',
    'Median_Gross_Rent': 'B25031_001E'
   
}

#Inverts the dictionary so the columns can be renamed, add keys from the forthcoming index dictionary to this dictionary
inv_census_codes_2015 = {v: k for k, v in census_codes_2015.items()}
inv_census_codes_2015.update({'District_Name':'District_Name', 'CD': 'CD', 'State_Id': 'State_Id','State': 'State', 'CD_Id_Year': 'CD_Id_Year'})


#Creates a string of codes to be used in the API call
columns_url_2015 = ''

for key in census_codes_2015:
    columns_url_2015 += census_codes_2015[key] + ','
    
columns_url_2015

'B01003_001E,B01001_002E,B01001_026E,B01002_001E,B21001_002E,B02001_002E,B02001_003E,B02001_005E,B02001_004E,C03001_006E,C03001_005E,C03001_007E,C03001_004E,B05001_002E,B05001_006E,B05012_003E,B09001_001E,B11001_003E,B06009_002E,B06009_005E,B15012_009E,B15012_010E,B28007_002E,B28007_003E,B28007_009E,B19013_001E,B06011_001E,B17001_002E,B19001_017E,B19083_001E,B23013_001E,B24050_001E,B24050_002E,B24050_005E,B24050_006E,B24050_008E,B24050_009E,B24050_013E,B24050_021E,B24050_022E,B24050_025E,B24050_027E,B24050_012E,B25031_001E,'

In [3]:
#Creates a dictionary of codes and corresponding titles for the 2019 American Community Survey
census_codes_2019 = {
    'Total_Pop': 'B01003_001E',
    'Total_Males': 'B01001_002E',
    'Total_Females': 'B01001_026E',
    'Median_Age': 'B01002_001E',
    'Total_Veterans': 'B21001_002E',
    'Total_White_People': 'B02001_002E',
    'Total_Afr_Am_People': 'B02001_003E',
    'Total_Asian': 'B02001_005E',
    'Total_American_Ind_Alk_Ntv': 'B02001_004E',
    'Total_Cuban_Origin': 'C03001_006E',
    'Total_Puerto_Rican_Origin':'C03001_005E',
    'Total_Dominican_Origin':'C03001_007E',
    'Total_Mexican_Origin': 'C03001_004E',
    'Total_Native_US':'B05001_002E',
    'Total_Non_Citizens': 'B05001_006E',
    'Total_Foreign_Born': 'B05012_003E',
    'Total_Children': 'B09001_001E',
    'Total_Married_Households': 'B11001_003E',
    'Total_Less_Highschool': 'B06009_002E',
    'Total_Bachelor_Holders': 'B06009_005E',
    'Total_STEM': 'B15012_009E',
    'Total_Business_Degree': 'B15012_010E',
    'Total_Labor_Force_Eligible': 'B28007_002E', #code changed in 2015
    'Total_Employed': 'B28007_003E',
    'Total_Unemployed': 'B28007_009E',
    'Median_Household_Income': 'B19013_001E',
    'Median_Income': 'B06011_001E',
    'Below_Poverty_Level_LTM': 'B17001_002E',
    'Total_Wealthy_Households': 'B19001_017E', #Earns $200k or more
    'Gini_Index': 'B19083_001E',
    'Median_Age_Of_Worker': 'B23013_001E',
    'Total_Workers': 'B24050_001E',
    'Total_Working_Agricult': 'B24050_002E',
    'Total_Working_Construction': 'B24050_005E',
    'Total_Working_Manufacturing': 'B24050_006E',
    'Total_Working_Retail': 'B24050_008E',
    'Total_Working_Transportation': 'B24050_009E',
    'Total_Working_Finance': 'B24050_013E',
    'Total_Working_Education': 'B24050_021E',
    'Total_Working_Health': 'B24050_022E',
    'Total_Working_Food': 'B24050_025E',
    'Total_Working_Public_Admin': 'B24050_027E',
    'Total_Working_Information': 'B24050_012E',
    'Median_Gross_Rent': 'B25031_001E'
   
}

#Inverts the dictionary so the columns can be renamed, add keys from the forthcoming index dictionary to this dictionary
inv_census_codes_2019 = {v: k for k, v in census_codes_2019.items()}
inv_census_codes_2019.update({'District_Name':'District_Name', 'CD': 'CD', 'State_Id': 'State_Id','State': 'State', 'CD_Id_Year': 'CD_Id_Year'})


#Creates a string of codes to be used in the API call
columns_url_2019 = ''

for key in census_codes_2019:
    columns_url_2019 += census_codes_2019[key] + ','
    
columns_url_2019

'B01003_001E,B01001_002E,B01001_026E,B01002_001E,B21001_002E,B02001_002E,B02001_003E,B02001_005E,B02001_004E,C03001_006E,C03001_005E,C03001_007E,C03001_004E,B05001_002E,B05001_006E,B05012_003E,B09001_001E,B11001_003E,B06009_002E,B06009_005E,B15012_009E,B15012_010E,B28007_002E,B28007_003E,B28007_009E,B19013_001E,B06011_001E,B17001_002E,B19001_017E,B19083_001E,B23013_001E,B24050_001E,B24050_002E,B24050_005E,B24050_006E,B24050_008E,B24050_009E,B24050_013E,B24050_021E,B24050_022E,B24050_025E,B24050_027E,B24050_012E,B25031_001E,'

In [4]:
#A function for the API call
def obtain_census_data(year, codes):
    congressional_district_url = 'https://api.census.gov/data/{}/acs/acs1?get={}NAME&for=congressional%20district:*&in=state:*'.format(year, codes)
    congressional_district_content = requests.get(congressional_district_url)
    congressional_district_contenttext = congressional_district_content.text.replace('\n', '').replace('null', '"Replace"')
    congressional_district_information_list = ast.literal_eval(congressional_district_contenttext)
    print(len(congressional_district_information_list))
    return congressional_district_information_list
    

In [5]:
#A function to turn the inner list from the API call into a dictionary
#The dictionary will represent a district and its census info
def turn_info_list_to_dict(keys_list, list_of_cd_info):
    return dict(zip(keys_list, list_of_cd_info))

In [6]:
#A function that takes in the list of lists and returns a list of dictionaries
def congressional_district_dictionary_list(list_of_cds_info):
    keys_list = list_of_cds_info[0]
    cd_dictionary_list = []
    
    for district in list_of_cds_info[1:]:
        cd_dictionary_list.append(turn_info_list_to_dict(keys_list, district))
    
    return cd_dictionary_list

In [7]:
#The code to obtain the 2015 data that will be used to predict the 2016 results 
congressional_data_2016 = congressional_district_dictionary_list(obtain_census_data('2015', columns_url_2015))

438


In [8]:
#The code to obtain the 2019 data that will be used to predict the 2020 results 
congressional_data_2020 = congressional_district_dictionary_list(obtain_census_data('2019', columns_url_2019))

438


In [9]:
#A dictionary of state codes and abbreviations to relate the tables that use either
#the abbreviation or full state name
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

#A dictionary to invert the above dictionary
inv_state = {v: k for k, v in states.items()}

In [10]:
#The following code creates a pandas dataframe from the 2015 API call then renames the
#columns to include underscores. A unique identifier for each district and election
#year is created
congressional_df_2016 = pd.DataFrame(congressional_data_2016)
congressional_df_2016.rename(columns = {'state': 'State_Id', 'NAME': 'District_Name', 'congressional district': 'CD'}, inplace = True)
congressional_df_2016['State'] = congressional_df_2016.District_Name.apply(lambda x: x.split(', ')[1])
congressional_df_2016['State'] = congressional_df_2016['State'].map(inv_state)
congressional_df_2016.loc[congressional_df_2016['CD'] == '00', 'CD'] = 'AL'
congressional_df_2016["CD_Id_Year"] = congressional_df_2016["State"] + '-' + congressional_df_2016["CD"] + '-2016'
congressional_df_2016.columns = congressional_df_2016.columns.to_series().map(inv_census_codes_2015)
congressional_df_2016.head()

,Total_Pop,Total_Males,Total_Females,Median_Age,Total_Veterans,Total_White_People,Total_Afr_Am_People,Total_Asian,Total_American_Ind_Alk_Ntv,Total_Cuban_Origin,...,Total_Working_Health,Total_Working_Food,Total_Working_Public_Admin,Total_Working_Information,Median_Gross_Rent,District_Name,State_Id,CD,State,CD_Id_Year
0,761778,368300,393478,40.0,35188,506183,91797,74832,2908,3075,...,47765,27220,46484,11334,1622,"Congressional District 8 (114th Congress), Mar...",24,08,MD,MD-08-2016
1,736190,354793,381397,41.0,45064,626968,46172,14655,1892,841,...,65798,24188,13941,6830,868,"Congressional District 1 (114th Congress), Mas...",25,01,MA,MA-01-2016
2,744886,363964,380922,39.1,42944,625647,39694,39563,1766,1190,...,63108,26636,12590,8010,1013,"Congressional District 2 (114th Congress), Mas...",25,02,MA,MA-02-2016
3,754068,372202,381866,39.0,37037,587198,25736,65457,1391,693,...,57824,27290,13154,10411,1085,"Congressional District 3 (114th Congress), Mas...",25,03,MA,MA-03-2016
4,750331,364844,385487,40.9,31741,652907,21406,44196,256,2405,...,64504,18434,12607,9701,1221,"Congressional District 4 (114th Congress), Mas...",25,04,MA,MA-04-2016


In [11]:
#The following code creates a pandas dataframe from the 2015 API call then renames the
#columns to include underscores. A unique identifier for each district and election
#year is created
congressional_df_2020 = pd.DataFrame(congressional_data_2020)
congressional_df_2020.rename(columns = {'state': 'State_Id', 'NAME': 'District_Name', 'congressional district': 'CD'}, inplace = True)
congressional_df_2020['State'] = congressional_df_2020.District_Name.apply(lambda x: x.split(', ')[1])
congressional_df_2020['State'] = congressional_df_2020['State'].map(inv_state)
congressional_df_2020.loc[congressional_df_2020['CD'] == '00', 'CD'] = 'AL'
congressional_df_2020["CD_Id_Year"] = congressional_df_2020["State"] + '-' + congressional_df_2020["CD"] + '-2020'
congressional_df_2020.columns = congressional_df_2020.columns.to_series().map(inv_census_codes_2019)
congressional_df_2020.head()

,Total_Pop,Total_Males,Total_Females,Median_Age,Total_Veterans,Total_White_People,Total_Afr_Am_People,Total_Asian,Total_American_Ind_Alk_Ntv,Total_Cuban_Origin,...,Total_Working_Health,Total_Working_Food,Total_Working_Public_Admin,Total_Working_Information,Median_Gross_Rent,District_Name,State_Id,CD,State,CD_Id_Year
0,717438,345865,371573,40.4,52884,479955,196641,10157,7360,1173,...,39790,23224,16966,3146,872,"Congressional District 1 (116th Congress), Ala...",01,01,AL,AL-01-2020
1,674920,325647,349273,39.0,51418,424115,217508,8458,2098,461,...,40314,23137,22557,3100,815,"Congressional District 2 (116th Congress), Ala...",01,02,AL,AL-02-2020
2,717896,349635,368261,38.9,46500,494456,188613,13239,2142,159,...,33235,22169,18617,3645,771,"Congressional District 3 (116th Congress), Ala...",01,03,AL,AL-03-2020
3,687453,337329,350124,41.1,38159,596835,50846,3259,3277,626,...,40598,20575,10059,4330,661,"Congressional District 4 (116th Congress), Ala...",01,04,AL,AL-04-2020
4,735858,358825,377033,39.7,55027,552016,134999,12180,4005,1053,...,38608,25089,23010,4871,783,"Congressional District 5 (116th Congress), Ala...",01,05,AL,AL-05-2020


In [12]:
#This cell concats the 2016 election data and 2020 election data
congressional_df = pd.concat([congressional_df_2016,congressional_df_2020], sort=False)
congressional_df

,Total_Pop,Total_Males,Total_Females,Median_Age,Total_Veterans,Total_White_People,Total_Afr_Am_People,Total_Asian,Total_American_Ind_Alk_Ntv,Total_Cuban_Origin,...,Total_Working_Health,Total_Working_Food,Total_Working_Public_Admin,Total_Working_Information,Median_Gross_Rent,District_Name,State_Id,CD,State,CD_Id_Year
0,761778,368300,393478,40.0,35188,506183,91797,74832,2908,3075,...,47765,27220,46484,11334,1622,"Congressional District 8 (114th Congress), Mar...",24,08,MD,MD-08-2016
1,736190,354793,381397,41.0,45064,626968,46172,14655,1892,841,...,65798,24188,13941,6830,868,"Congressional District 1 (114th Congress), Mas...",25,01,MA,MA-01-2016
2,744886,363964,380922,39.1,42944,625647,39694,39563,1766,1190,...,63108,26636,12590,8010,1013,"Congressional District 2 (114th Congress), Mas...",25,02,MA,MA-02-2016
3,754068,372202,381866,39.0,37037,587198,25736,65457,1391,693,...,57824,27290,13154,10411,1085,"Congressional District 3 (114th Congress), Mas...",25,03,MA,MA-03-2016
4,750331,364844,385487,40.9,31741,652907,21406,44196,256,2405,...,64504,18434,12607,9701,1221,"Congressional District 4 (114th Congress), Mas...",25,04,MA,MA-04-2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,715828,357863,357965,42.0,41769,655759,14016,18607,3908,317,...,51426,21488,11984,4591,785,"Congressional District 6 (116th Congress), Wis...",55,06,WI,WI-06-2020
433,714544,360413,354131,45.0,49227,665940,5745,11811,14804,470,...,56288,19799,11666,3133,728,"Congressional District 7 (116th Congress), Wis...",55,07,WI,WI-07-2020
434,736079,367415,368664,40.8,41751,655069,12180,18059,19406,1031,...,52790,21002,13298,3877,771,"Congressional District 8 (116th Congress), Wis...",55,08,WI,WI-08-2020
435,578759,296005,282754,38.1,41390,525860,7116,4633,13774,103,...,38329,21117,16995,3236,822,Congressional District (at Large) (116th Congr...,56,AL,WY,WY-AL-2020


In [13]:
#This code imports the data for the previous presidential elections and renames the columns
#A unique identifier for each district and election year is created
election_results = pd.read_csv('raw_data_2020/election_results.csv')
election_results['State_AB'] = election_results.CD.apply(lambda x: x.split('-')[0])
election_results.rename(columns = {'CD': 'CD_Id'}, inplace = True)
election_results["CD_Id_Year"] = election_results['CD_Id'] + '-' + election_results['Year'].astype(str)
election_results.drop(['CD_Id', 'Year', 'State_AB'], axis = 1, inplace = True)
election_results = election_results[['CD_Id_Year', 'Previous_Presidential_D', 'Previous_Presidential_R', 'Target_Presidential_D', 'Target_Presidential_R']]
election_results

,CD_Id_Year,Previous_Presidential_D,Previous_Presidential_R,Target_Presidential_D,Target_Presidential_R
0,AK-AL-2016,41.2,55.3,37.6,52.8
1,AL-01-2016,37.4,61.8,34.1,63.5
2,AL-02-2016,36.4,62.9,33.0,64.9
3,AL-03-2016,36.8,62.3,32.3,65.3
4,AL-04-2016,24.0,74.8,17.4,80.4
...,...,...,...,...,...
865,WI-08-2020,38.6,56.2,41.3,57.2
866,WV-01-2020,26.4,68.0,30.3,68.0
867,WV-02-2020,29.4,65.8,32.8,65.4
868,WV-03-2020,23.3,72.5,25.5,73.1


In [14]:
#This code combines the census data and election results
election_census = pd.merge(congressional_df, election_results, on='CD_Id_Year', how = 'left')
election_census.shape[0]

,Total_Pop,Total_Males,Total_Females,Median_Age,Total_Veterans,Total_White_People,Total_Afr_Am_People,Total_Asian,Total_American_Ind_Alk_Ntv,Total_Cuban_Origin,...,Median_Gross_Rent,District_Name,State_Id,CD,State,CD_Id_Year,Previous_Presidential_D,Previous_Presidential_R,Target_Presidential_D,Target_Presidential_R
0,761778,368300,393478,40.0,35188,506183,91797,74832,2908,3075,...,1622,"Congressional District 8 (114th Congress), Mar...",24,08,MD,MD-08-2016,62.0,36.0,65.0,30.6
1,736190,354793,381397,41.0,45064,626968,46172,14655,1892,841,...,868,"Congressional District 1 (114th Congress), Mas...",25,01,MA,MA-01-2016,64.0,34.3,57.2,36.5
2,744886,363964,380922,39.1,42944,625647,39694,39563,1766,1190,...,1013,"Congressional District 2 (114th Congress), Mas...",25,02,MA,MA-02-2016,58.7,39.2,56.2,36.8
3,754068,372202,381866,39.0,37037,587198,25736,65457,1391,693,...,1085,"Congressional District 3 (114th Congress), Mas...",25,03,MA,MA-03-2016,56.9,41.4,58.2,35.4
4,750331,364844,385487,40.9,31741,652907,21406,44196,256,2405,...,1221,"Congressional District 4 (114th Congress), Mas...",25,04,MA,MA-04-2016,57.2,41.3,59.2,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,715828,357863,357965,42.0,41769,655759,14016,18607,3908,317,...,785,"Congressional District 6 (116th Congress), Wis...",55,06,WI,WI-06-2020,38.8,55.7,41.6,56.8
870,714544,360413,354131,45.0,49227,665940,5745,11811,14804,470,...,728,"Congressional District 7 (116th Congress), Wis...",55,07,WI,WI-07-2020,37.3,57.7,39.3,59.2
871,736079,367415,368664,40.8,41751,655069,12180,18059,19406,1031,...,771,"Congressional District 8 (116th Congress), Wis...",55,08,WI,WI-08-2020,38.6,56.2,41.3,57.2
872,578759,296005,282754,38.1,41390,525860,7116,4633,13774,103,...,822,Congressional District (at Large) (116th Congr...,56,AL,WY,WY-AL-2020,22.5,70.1,26.7,70.4


In [15]:
#This line checks for any null values in the target column
#Some districts became obsolete between the 2016 and 2020 election, creating null values
election_census.Target_Presidential_R.isnull().sum(axis = 0)

70

In [16]:
#This line views any row with null values in the target column
election_census[election_census.isnull().any(axis=1)].head()

,Total_Pop,Total_Males,Total_Females,Median_Age,Total_Veterans,Total_White_People,Total_Afr_Am_People,Total_Asian,Total_American_Ind_Alk_Ntv,Total_Cuban_Origin,...,Median_Gross_Rent,District_Name,State_Id,CD,State,CD_Id_Year,Previous_Presidential_D,Previous_Presidential_R,Target_Presidential_D,Target_Presidential_R
247,3474182,1657467,1816715,40.0,83847,2355164,341710,7264,10984,12824,...,433,Resident Commissioner District (at Large) (114...,72,98,PR,PR-98-2016,NaN,NaN,NaN,NaN
335,672228,319705,352523,33.8,25963,269143,318831,25944,1661,4021,...,1417,"Delegate District (at Large) (114th Congress),...",11,98,DC,DC-98-2016,NaN,NaN,NaN,NaN
437,717438,345865,371573,40.4,52884,479955,196641,10157,7360,1173,...,872,"Congressional District 1 (116th Congress), Ala...",01,01,AL,AL-01-2020,34.1,63.5,NaN,NaN
438,674920,325647,349273,39.0,51418,424115,217508,8458,2098,461,...,815,"Congressional District 2 (116th Congress), Ala...",01,02,AL,AL-02-2020,33.0,64.9,NaN,NaN
439,717896,349635,368261,38.9,46500,494456,188613,13239,2142,159,...,771,"Congressional District 3 (116th Congress), Ala...",01,03,AL,AL-03-2020,32.3,65.3,NaN,NaN


In [17]:
#This line reassigns the dataframe to ensure no rows contain null values in the target column
election_census = election_census[pd.notnull(election_census['Target_Presidential_R'])]

In [18]:
election_census.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 0 to 872
Data columns (total 53 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Total_Pop                     804 non-null    object 
 1   Total_Males                   804 non-null    object 
 2   Total_Females                 804 non-null    object 
 3   Median_Age                    804 non-null    object 
 4   Total_Veterans                804 non-null    object 
 5   Total_White_People            804 non-null    object 
 6   Total_Afr_Am_People           804 non-null    object 
 7   Total_Asian                   804 non-null    object 
 8   Total_American_Ind_Alk_Ntv    804 non-null    object 
 9   Total_Cuban_Origin            804 non-null    object 
 10  Total_Puerto_Rican_Origin     804 non-null    object 
 11  Total_Dominican_Origin        804 non-null    object 
 12  Total_Mexican_Origin          804 non-null    object 
 13  Total

In [ ]:
#This line finds any cell that equals 'Replace' and prints the row
#election_census[election_census.eq('Replace').any(1)]

In [20]:
#Saves the main data frame
election_census.to_csv('congressional_data_and_targets.csv', index = False)